In [30]:
from  bs4 import  BeautifulSoup 
import requests
import re
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)


#rr['days_survived'] = rr.apply(checkdays, axis=1)

def add_0(val):
    return str(val).zfill(4)

In [31]:
performance = pd.read_csv('/home/analytics/Survivor/SurvivalScores.csv',index_col=0)
castaways = pd.read_csv('/home/analytics/Survivor/castaways.csv',index_col=0)
alliance = pd.read_csv('/home/analytics/Survivor/AllianceScores.csv',index_col=0)
alliance = alliance.drop(['name','season_number'],axis=1)
alliance

,Alliance_Score
ID,
116,-1.000000
115,-1.000000
114,-1.000000
113,-1.000000
112,-1.000000
111,-1.000000
110,0.500000
109,0.262500
108,0.487500


In [32]:
#performance.columns.values
for col in performance.columns.values:
    if col != 'Contestant':
        performance[col] = pd.to_numeric(performance[col],errors='coerce')
        
performance

,Contestant,ChW,ChA,ChW%,VFB,VAP,TotV,TCA,TC%,wTCR,JVF,TotJ,JV%,SurvSc,SurvAv
ID,,,,,,,,,,,,,,,
101,Richard Hatch,1.87,15.10,0.12,10.0,6.0,73.0,11.0,0.90,2.55,4.0,7.0,0.570,1.60,7.84
102,Kelly Wiglesworth,5.87,15.10,0.39,6.0,0.0,73.0,11.0,0.55,3.82,3.0,7.0,0.430,1.36,12.26
103,Rudy Boesch,1.62,15.10,0.11,10.0,8.0,73.0,11.0,0.90,2.12,NaN,NaN,NaN,1.01,3.74
104,Susan Hawk,0.87,14.10,0.06,9.0,5.0,70.0,10.0,0.89,2.80,NaN,NaN,NaN,0.95,3.67
105,Sean Kenniff,1.87,13.10,0.14,6.0,9.0,66.0,9.0,0.65,1.44,NaN,NaN,NaN,0.79,3.30
106,Colleen Haskell,1.56,11.07,0.14,3.0,7.0,61.0,8.0,0.36,0.95,NaN,NaN,NaN,0.50,2.51
107,Gervase Peterson,2.89,9.07,0.32,1.0,6.0,55.0,7.0,0.13,0.40,NaN,NaN,NaN,0.45,3.29
108,Jenna Lewis,1.23,7.07,0.17,4.0,11.0,48.0,6.0,0.63,1.24,NaN,NaN,NaN,0.80,2.47
109,Greg Buis,2.56,5.07,0.50,1.0,6.0,40.0,5.0,0.17,0.56,NaN,NaN,NaN,0.67,3.12


In [33]:
castaways

,name,age,gender,state,season,place,days_survived,votes_against,place_percent,time_playing,total_times_played,players_in_season,season_number,is_censored
ID,,,,,,,,,,,,,,
116,Sonja Christopher,63,Female,CA,Borneo,16,3,4,0.062500,1,1,16,1,0
115,B.B. Andersen,64,Male,KS,Borneo,15,6,6,0.125000,1,1,16,1,0
114,Stacey Stillman,27,Female,CA,Borneo,14,9,6,0.187500,1,1,16,1,0
113,Ramona Gray,29,Female,NJ,Borneo,13,12,6,0.250000,1,1,16,1,0
112,Dirk Been,23,Male,WI,Borneo,12,15,4,0.312500,1,1,16,1,0
111,Joel Klug,27,Male,AR,Borneo,11,18,4,0.375000,1,1,16,1,0
110,Gretchen Cordy,38,Female,TN,Borneo,10,21,4,0.437500,1,1,16,1,0
109,Greg Buis,24,Male,CO,Borneo,9,24,6,0.500000,1,1,16,1,0
108,Jenna Lewis,22,Female,NH,Borneo,8,27,11,0.562500,1,2,16,1,0


In [34]:
mid = castaways.join(performance)
combined = mid.join(alliance)
combined.index = combined.index.map(add_0)
combined = combined.drop('Contestant',axis=1)
combined

,name,age,gender,state,season,place,days_survived,votes_against,place_percent,time_playing,total_times_played,players_in_season,season_number,is_censored,ChW,ChA,ChW%,VFB,VAP,TotV,TCA,TC%,wTCR,JVF,TotJ,JV%,SurvSc,SurvAv,Alliance_Score
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0101,Richard Hatch,39,Male,RI,Borneo,1,39,6,1.000000,1,2,16,1,1,1.87,15.10,0.12,10.0,6.0,73.0,11.0,0.90,2.55,4.0,7.0,0.570,1.60,7.84,0.688889
0102,Kelly Wiglesworth,22,Female,NV,Borneo,2,39,0,0.937500,1,2,16,1,1,5.87,15.10,0.39,6.0,0.0,73.0,11.0,0.55,3.82,3.0,7.0,0.430,1.36,12.26,0.413535
0103,Rudy Boesch,72,Male,VA,Borneo,3,38,8,0.875000,1,2,16,1,0,1.62,15.10,0.11,10.0,8.0,73.0,11.0,0.90,2.12,NaN,NaN,NaN,1.01,3.74,0.753030
0104,Susan Hawk,38,Female,WI,Borneo,4,37,7,0.812500,1,2,16,1,0,0.87,14.10,0.06,9.0,5.0,70.0,10.0,0.89,2.80,NaN,NaN,NaN,0.95,3.67,0.741667
0105,Sean Kenniff,30,Male,NY,Borneo,5,36,9,0.750000,1,1,16,1,0,1.87,13.10,0.14,6.0,9.0,66.0,9.0,0.65,1.44,NaN,NaN,NaN,0.79,3.30,0.527778
0106,Colleen Haskell,23,Female,FL,Borneo,6,33,7,0.687500,1,1,16,1,0,1.56,11.07,0.14,3.0,7.0,61.0,8.0,0.36,0.95,NaN,NaN,NaN,0.50,2.51,0.517619
0107,Gervase Peterson,30,Male,NJ,Borneo,7,30,6,0.625000,1,2,16,1,0,2.89,9.07,0.32,1.0,6.0,55.0,7.0,0.13,0.40,NaN,NaN,NaN,0.45,3.29,0.338690
0108,Jenna Lewis,22,Female,NH,Borneo,8,27,11,0.562500,1,2,16,1,0,1.23,7.07,0.17,4.0,11.0,48.0,6.0,0.63,1.24,NaN,NaN,NaN,0.80,2.47,0.487500
0109,Greg Buis,24,Male,CO,Borneo,9,24,6,0.500000,1,1,16,1,0,2.56,5.07,0.50,1.0,6.0,40.0,5.0,0.17,0.56,NaN,NaN,NaN,0.67,3.12,0.262500


In [35]:
returnee = combined['total_times_played']>1
returnees = combined[returnee]
returnees

,name,age,gender,state,season,place,days_survived,votes_against,place_percent,time_playing,total_times_played,players_in_season,season_number,is_censored,ChW,ChA,ChW%,VFB,VAP,TotV,TCA,TC%,wTCR,JVF,TotJ,JV%,SurvSc,SurvAv,Alliance_Score
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0101,Richard Hatch,39,Male,RI,Borneo,1,39,6,1.000000,1,2,16,1,1,1.87,15.10,0.12,10.0,6.0,73.0,11.0,0.90,2.55,4.0,7.0,0.570,1.60,7.84,0.688889
0102,Kelly Wiglesworth,22,Female,NV,Borneo,2,39,0,0.937500,1,2,16,1,1,5.87,15.10,0.39,6.0,0.0,73.0,11.0,0.55,3.82,3.0,7.0,0.430,1.36,12.26,0.413535
0103,Rudy Boesch,72,Male,VA,Borneo,3,38,8,0.875000,1,2,16,1,0,1.62,15.10,0.11,10.0,8.0,73.0,11.0,0.90,2.12,NaN,NaN,NaN,1.01,3.74,0.753030
0104,Susan Hawk,38,Female,WI,Borneo,4,37,7,0.812500,1,2,16,1,0,0.87,14.10,0.06,9.0,5.0,70.0,10.0,0.89,2.80,NaN,NaN,NaN,0.95,3.67,0.741667
0107,Gervase Peterson,30,Male,NJ,Borneo,7,30,6,0.625000,1,2,16,1,0,2.89,9.07,0.32,1.0,6.0,55.0,7.0,0.13,0.40,NaN,NaN,NaN,0.45,3.29,0.338690
0108,Jenna Lewis,22,Female,NH,Borneo,8,27,11,0.562500,1,2,16,1,0,1.23,7.07,0.17,4.0,11.0,48.0,6.0,0.63,1.24,NaN,NaN,NaN,0.80,2.47,0.487500
0201,Tina Wesson,39,Female,TN,The Australian Outback,1,42,0,1.000000,1,3,16,2,1,0.59,13.09,0.05,11.0,0.0,73.0,11.0,1.00,7.00,4.0,7.0,0.570,1.62,11.02,0.539842
0202,Colby Donaldson,26,Male,TX,The Australian Outback,2,42,9,0.937500,1,3,16,2,1,8.09,13.09,0.62,10.0,5.0,73.0,11.0,0.90,2.83,3.0,7.0,0.430,1.95,13.49,0.500959
0206,Amber Brkich,22,Female,PA,The Australian Outback,6,33,6,0.687500,1,2,16,2,0,0.59,9.09,0.07,5.0,6.0,61.0,8.0,0.61,1.75,NaN,NaN,NaN,0.68,2.34,0.705556


In [36]:
mid = returnees.groupby('name').mean()
mid = mid.drop(['is_censored','time_playing'],axis=1)
mid = mid.round(4)
mid['gender'] = None
mid['state'] = None
mid['seasons_played'] = None
mid['prev_ids'] = None
mid['new_id'] = None
mid['min_place'] = None

In [37]:
#id_num = 0
#newid = 'R'+str(id_num)
def strbuild(lst):
    string = ''
    for z in range(len(lst)):
        if z > 0:
            string+=', '
        string+=lst[z]
    return str(string)


mid2 = mid.copy(deep=True)
for x in range(len(mid2)):
    #print(index)
    gender = ''
    state=''
    new_id = 1000
    min_place=1000
    seasons=[]
    ids=[]
    for y in range(len(returnees)):
        if str(mid2.iloc[x].name) == str(returnees.iloc[y,0]):
            #print(index,str(rowR[0]))
            gender = returnees.iloc[y,2]
            state = returnees.iloc[y,3]
            seasons.append(returnees.iloc[y,4])
            ids.append(returnees.iloc[y].name)
            new_id = min(new_id,returnees.iloc[y,12])
            min_place = min(min_place,returnees.iloc[y,5])
    mid2.iloc[x,-6] = gender
    mid2.iloc[x,-5] = state
    mid2.iloc[x,-4] = strbuild(seasons)
    mid2.iloc[x,-3] = strbuild(ids)
    mid2.iloc[x,-2] = new_id
    mid2.iloc[x,-1] = min_place
    
mid2.sort_values(['new_id','min_place'],inplace=True)
mid2 = mid2.drop('min_place',axis=1)

id_num = 0
for x in range(len(mid2)):
    id_num += 1
    newid = 'R'+str(id_num).zfill(3)
    mid2.iloc[x,-1] = newid
    
mid2.reset_index(level=0, inplace=True)
#mid2.set_index('new_id',inplace=True)

In [38]:
returnees_avg = mid2.copy(deep=True)
#add columns to allow for merging
returnees_avg['ID']=returnees_avg['new_id']
combined['ID']=combined.index
c = list(combined.columns.values)
r = list(returnees_avg.columns.values)
#print(c,'\n',r)
rc = list(set(c)^set(r))
#print(rc)
for x in rc:
    #print(x)
    if x in c:
        #print('c',x)
        if x == 'season':
            returnees_avg[x] = None
        elif x == 'is_censored':
            returnees_avg[x] = 1
        else:
            returnees_avg[x] = np.nan
    if x in r:
        #print('r',x)
        combined[x] = None

returnees_avg['ID']=returnees_avg['ID'].astype('str')
        
#for z in combined.columns.values:
#    print(z,combined[z].dtype,returnees_avg[z].dtype)
        

#mid['state'].astype('str')
#mid['seasons_played'].astype('str')
#mid['prev_ids'].astype('str')
#mid['new_id'].astype('str')
#mid['min_place'].astype('str')
#print(combined.dtypes,'\n',returnees_avg.dtypes)
combined_with_returnee = combined.merge(returnees_avg,how='outer')

/home/analytics/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: FutureWarning: 'ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
/home/analytics/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:970: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


In [39]:
combined_with_returnee['FTC'] = 0
combined_with_returnee.loc[combined_with_returnee['TotJ'] > 0, 'FTC'] = 1

combined_with_returnee.set_index('ID',inplace=True)
combined_with_returnee.to_csv('/home/analytics/Survivor/FullContestantData.csv', index=True)

In [40]:
combined_with_returnee

,name,age,gender,state,season,place,days_survived,votes_against,place_percent,time_playing,total_times_played,players_in_season,season_number,is_censored,ChW,ChA,ChW%,VFB,VAP,TotV,TCA,TC%,wTCR,JVF,TotJ,JV%,SurvSc,SurvAv,Alliance_Score,prev_ids,new_id,seasons_played,FTC
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0101,Richard Hatch,39.0000,Male,RI,Borneo,1.0000,39.0000,6.0000,1.000000,1.0,2,16.0000,1.0000,1,1.8700,15.1000,0.1200,10.0000,6.0000,73.0000,11.0000,0.9000,2.5500,4.0,7.0,0.570,1.6000,7.8400,0.688889,None,None,None,1
0102,Kelly Wiglesworth,22.0000,Female,NV,Borneo,2.0000,39.0000,0.0000,0.937500,1.0,2,16.0000,1.0000,1,5.8700,15.1000,0.3900,6.0000,0.0000,73.0000,11.0000,0.5500,3.8200,3.0,7.0,0.430,1.3600,12.2600,0.413535,None,None,None,1
0103,Rudy Boesch,72.0000,Male,VA,Borneo,3.0000,38.0000,8.0000,0.875000,1.0,2,16.0000,1.0000,0,1.6200,15.1000,0.1100,10.0000,8.0000,73.0000,11.0000,0.9000,2.1200,NaN,NaN,NaN,1.0100,3.7400,0.753030,None,None,None,0
0104,Susan Hawk,38.0000,Female,WI,Borneo,4.0000,37.0000,7.0000,0.812500,1.0,2,16.0000,1.0000,0,0.8700,14.1000,0.0600,9.0000,5.0000,70.0000,10.0000,0.8900,2.8000,NaN,NaN,NaN,0.9500,3.6700,0.741667,None,None,None,0
0105,Sean Kenniff,30.0000,Male,NY,Borneo,5.0000,36.0000,9.0000,0.750000,1.0,1,16.0000,1.0000,0,1.8700,13.1000,0.1400,6.0000,9.0000,66.0000,9.0000,0.6500,1.4400,NaN,NaN,NaN,0.7900,3.3000,0.527778,None,None,None,0
0106,Colleen Haskell,23.0000,Female,FL,Borneo,6.0000,33.0000,7.0000,0.687500,1.0,1,16.0000,1.0000,0,1.5600,11.0700,0.1400,3.0000,7.0000,61.0000,8.0000,0.3600,0.9500,NaN,NaN,NaN,0.5000,2.5100,0.517619,None,None,None,0
0107,Gervase Peterson,30.0000,Male,NJ,Borneo,7.0000,30.0000,6.0000,0.625000,1.0,2,16.0000,1.0000,0,2.8900,9.0700,0.3200,1.0000,6.0000,55.0000,7.0000,0.1300,0.4000,NaN,NaN,NaN,0.4500,3.2900,0.338690,None,None,None,0
0108,Jenna Lewis,22.0000,Female,NH,Borneo,8.0000,27.0000,11.0000,0.562500,1.0,2,16.0000,1.0000,0,1.2300,7.0700,0.1700,4.0000,11.0000,48.0000,6.0000,0.6300,1.2400,NaN,NaN,NaN,0.8000,2.4700,0.487500,None,None,None,0
0109,Greg Buis,24.0000,Male,CO,Borneo,9.0000,24.0000,6.0000,0.500000,1.0,1,16.0000,1.0000,0,2.5600,5.0700,0.5000,1.0000,6.0000,40.0000,5.0000,0.1700,0.5600,NaN,NaN,NaN,0.6700,3.1200,0.262500,None,None,None,0
